In [ ]:
from paramiko import SSHClient,AutoAddPolicy
from scp import SCPClient
import sys, pickle, os
ssh = SSHClient()
ssh.set_missing_host_key_policy(AutoAddPolicy())
ssh.load_system_host_keys()
ssh.connect('euler.ethz.ch',username='phornauer',password='')

In [ ]:
# Define progress callback that prints the current percentage completed for the file
def progress(filename, size, sent):
    sys.stdout.write("%s's progress: %.2f%%   \r" % (filename, float(sent)/float(size)*100) )

In [ ]:
# SCPCLient takes a paramiko transport and progress callback as its arguments.
scp = SCPClient(ssh.get_transport())

In [ ]:
cmd = "find /cluster/project/bsse_sdsc/BELUB/Measurements/Cell_line*/*/*/*etwork/ -name '*.raw.h5'"
stdin, stdout, stderr = ssh.exec_command(cmd) 

In [ ]:
file_path_list = []
recording_dates = ['190314','190321','190328','190404','190411',
                  '190909','190916','190923','190930','191007',
                  '200127','200203','200210','200217','200224']
for line in stdout.read().splitlines():
    recording_path = line.decode()
    if any(recording_date in line.decode() for recording_date in recording_dates):
        file_path_list.append(recording_path)


In [ ]:
with open("file_path_list.pkl", "wb") as output:
    pickle.dump(file_path_list, output)

In [ ]:
save_root = '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Mea1k/phornauer/DeePhysSortings'
path = file_path_list[0]
path_parts = path.split('/')

In [ ]:
remote_path = file_path_list[0]
path_parts = remote_path.split('/')
local_path = os.path.join(save_root, path_parts[7], path_parts[8])
os.makedirs(local_path,exist_ok=True)
scp.get(remote_path=remote_path, local_path=local_path)

In [ ]:
for file_idx, remote_path in enumerate(file_path_list):
    path_parts = remote_path.split('/')
    local_path = os.path.join(save_root, path_parts[7], path_parts[8])
    os.makedirs(local_path,exist_ok=True)
    scp.get(remote_path=remote_path, local_path=local_path)
    if (file_idx + 1)%10 == 0:
        print(f'Finished {file_idx+1}/{len(file_path_list)} recordings')

In [ ]:
import pickle
with open("file_path_list.pkl", "rb") as output:
    path_list = pickle.load(output)

In [ ]:
path_list